In [1]:
from bs4 import BeautifulSoup #Python Web scraping library
import requests # library to handle requests
import json # library to handle JSON files
import xml
import pandas as pd #Python library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.35 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.18 MB/s
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   

Collecting Toronto Neighborhoods and their Postal codes by web scraping

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(url).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')

In [4]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighborhood  = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighborhood_var  = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighborhood_var = str(td_cell.text).strip()
            tag_a_Neighborhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighborhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighborhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighborhood.append(Neighborhood_var)

In [5]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighborhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighborhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighborhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighborhood_u.append(n_var)

In [9]:
toronto_dict = {'Borough':Borough_u,'Postcode':Postcode_u,'Neighborhood':Neighborhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
#df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Borough,Neighborhood,Postcode
0,North York,Victoria Village,M4A
1,North York,"Silver Hills, York Mills",M2L
2,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P
3,Downtown Toronto,Berczy Park,M5E
4,Downtown Toronto,Ryerson,M5B


In [15]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_coords=df_data_1

In [18]:
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_toronto = pd.merge(df_toronto, df_coords.rename(columns={'Postal Code':'Postcode'}), on='Postcode',  how='left')

In [21]:
df_toronto.head(10)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Victoria Village,M4A,43.725882,-79.315572
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
2,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",M1P,43.757410,-79.273304
3,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
4,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
5,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
6,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,43.739416,-79.588437
7,Downtown Toronto,"Toronto Islands, Union Station",M5J,43.640816,-79.381752
8,North York,Bayview Village,M2K,43.786947,-79.385975
9,Etobicoke,"Humber Bay, King's Mill Park, Mimico NE, Old M...",M8Y,43.636258,-79.498509


Create a map of North York and its neighborhoods

In [22]:
address = 'North York,Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinate of North York ,Toronto are {}, {}.'.format(latitude_x, longitude_y))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of North York ,Toronto are 43.7709163, -79.4124102.


In [23]:
borough_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
borough_data.head(7)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Victoria Village,M4A,43.725882,-79.315572
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
2,North York,Bayview Village,M2K,43.786947,-79.385975
3,North York,Bedford Park,M5M,43.733283,-79.419750
4,North York,Downsview West,M3L,43.739015,-79.506944
5,North York,"Bathurst Manor, Wilson Heights",M3H,43.754328,-79.442259
6,North York,Humber Summit,M9L,43.756303,-79.565963


In [24]:
# create map of North York,Toronto using latitude and longitude values
map_scarb = folium.Map(location=[latitude_x, longitude_y], zoom_start=12)

In [26]:
# add markers to map
#for lat, lng, nei in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
for lat, lng, nei in zip(borough_data['Latitude'], borough_data['Longitude'], borough_data['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarb)  
    
map_scarb

In [27]:
CLIENT_ID = 'HB4BJSNOUSOLGGTW2FE2S4T4GMBC1WY3H5THCRCQ4QCR3UGU' # Foursquare ID
CLIENT_SECRET = 'OSOOXGGSMMHJSU12PYPB2T1XGI1LHYNVW0KLSXG2FV33NP50' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [28]:
borough_data = df_toronto[df_toronto['Borough'] == 'North York'].reset_index(drop=True)
borough_data

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,North York,Victoria Village,M4A,43.725882,-79.315572
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714
2,North York,Bayview Village,M2K,43.786947,-79.385975
3,North York,Bedford Park,M5M,43.733283,-79.419750
4,North York,Downsview West,M3L,43.739015,-79.506944
5,North York,"Bathurst Manor, Wilson Heights",M3H,43.754328,-79.442259
6,North York,Humber Summit,M9L,43.756303,-79.565963
7,North York,Maple Leaf Park,M6L,43.713756,-79.490074
8,North York,Hillcrest Village,M2H,43.803762,-79.363452
9,North York,"Emery, Humberlea",M9M,43.724766,-79.532242


In [29]:
address = 'North York,Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude
print('The geograpical coordinate of North York,Toronto are {}, {}.'.format(latitude_x, longitude_y))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of North York,Toronto are 43.7709163, -79.4124102.


In [30]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)

In [31]:
results = requests.get(url).json()

In [32]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.events.count', 'venue.events.summary', 'venue.id',
       'venue.location.address', 'venue.location.cc', 'venue.location.city',
       'venue.location.country', 'venue.location.crossStreet',
       'venue.location.distance', 'venue.location.formattedAddress',
       'venue.location.labeledLatLngs', 'venue.location.lat',
       'venue.location.lng', 'venue.location.neighborhood',
       'venue.location.postalCode', 'venue.location.state', 'venue.name',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,The Captain's Boil,"[{'shortName': 'Seafood', 'pluralName': 'Seafo...",43.773255,-79.413805
1,Aroma Espresso Bar,"[{'shortName': 'Café', 'pluralName': 'Cafés', ...",43.769449,-79.413081
2,Loblaws,"[{'shortName': 'Grocery Store', 'pluralName': ...",43.768648,-79.412597
3,Starbucks,"[{'shortName': 'Coffee Shop', 'pluralName': 'C...",43.768192,-79.413021
4,Burrito Boyz,"[{'shortName': 'Burritos', 'pluralName': 'Burr...",43.773054,-79.414082
5,The Keg,"[{'shortName': 'Steakhouse', 'pluralName': 'St...",43.766579,-79.412131
6,Konjiki Ramen,"[{'shortName': 'Ramen', 'pluralName': 'Ramen R...",43.766998,-79.412222
7,Cineplex Cinemas Empress Walk,"[{'shortName': 'Movie Theater', 'pluralName': ...",43.768625,-79.412613
8,Dakgogi,"[{'shortName': 'Korean', 'pluralName': 'Korean...",43.773010,-79.413875
9,Owl of Minerva,"[{'shortName': 'Korean', 'pluralName': 'Korean...",43.773328,-79.413915


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
ny_venues = getNearbyVenues(names=borough_data['Neighborhood'],
                                   latitudes=borough_data['Latitude'],
                                   longitudes=borough_data['Longitude']
                                  )

Victoria Village
Silver Hills, York Mills
Bayview Village
Bedford Park
Downsview West
Bathurst Manor, Wilson Heights
Humber Summit
Maple Leaf Park
Hillcrest Village
Emery, Humberlea
Newtonbrook, Willowdale
Northwood Park, York University
CFB Toronto
Lawrence Heights, Lawrence Manor
Willowdale West
Glencairn
Henry Farm
Parkwoods
Flemingdon Park


In [37]:
print(ny_venues.shape)

(261, 7)


In [39]:
ny_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights",20,20,20,20,20,20
Bayview Village,9,9,9,9,9,9
Bedford Park,29,29,29,29,29,29
CFB Toronto,4,4,4,4,4,4
Downsview West,8,8,8,8,8,8


In [40]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 110 uniques categories.


In [42]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Greek Restaurant,Grocery Store,Gym,Hardware Store,Health Food Store,Hockey Arena,Home Service,Housing Development,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Liquor Store,Massage Studio,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Nightclub,Office,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
ny_onehot.shape

(261, 111)

In [44]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bowling Alley,Bridal Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Greek Restaurant,Grocery Store,Gym,Hardware Store,Health Food Store,Hockey Arena,Home Service,Housing Development,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Liquor Store,Massage Studio,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Nightclub,Office,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Bathurst Manor, Wilson Heights",0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.050000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.100000,0.000000,0.05,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.000000,0.050000,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.05,0.0,0.0,0.000,0.0,0.0,0.0,0.100,0.0,0.050000,0.050000,0.0,0.0,0.000000,0.0,0.050000,0.0,0.050000,0.0,0.050,0.000000,0.000000,0.0,0.0,0.0,0.0,0.050000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.05,0.000,0.0,0.0,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.111111,0.0,0.0,0.0,0.111111,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.111111,0.0,0.000000,0.0,0.000,0.111111,0.111111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0
2,Bedford Park,0.0,0.00,0.034483,0.0,0.0,0.034483,0.034483,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.0,0.000000,0.0,0.103448,0.034483,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.034483,0.068966,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.034483,0.034483,0.0,0.034483,0.0,0.0,0.0,0.0,0.034483,0.103448,0.000000,0.0,0.068966,0.0,0.034483,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.034483,0.034483,0.0,0.0,0.034483,0.0,0.034483,0.0,0.034483,0.0,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.034483,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.00,0.000,0.0,0.0,0.0
3,CFB Toronto,0.0,0.25,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.250000,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.000,0.0,0.0,0.0,0.250,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.250000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0

In [45]:
ny_grouped.shape

(19, 111)

In [46]:
num_top_venues = 5
for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights----
              venue  freq
0       Coffee Shop  0.10
1              Park  0.10
2       Bridal Shop  0.05
3  Community Center  0.05
4     Deli / Bodega  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.22
1        Grocery Store  0.11
2   Chinese Restaurant  0.11
3           Restaurant  0.11
4  Japanese Restaurant  0.11


----Bedford Park----
                  venue  freq
0    Italian Restaurant  0.10
1           Coffee Shop  0.10
2             Juice Bar  0.07
3  Fast Food Restaurant  0.07
4         Grocery Store  0.03


----CFB Toronto----
            venue  freq
0         Airport  0.25
1     Coffee Shop  0.25
2            Park  0.25
3  Sandwich Place  0.25
4        Pharmacy  0.00


----Downsview West----
           venue  freq
0  Grocery Store  0.12
1           Bank  0.12
2    Pizza Place  0.12
3           Park  0.12
4  Shopping Mall  0.12


----Emery, Humberlea----
                venue  freq
0   Convenience St

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights",Park,Coffee Shop,Fried Chicken Joint,Diner,Middle Eastern Restaurant,Pharmacy,Pizza Place,Community Center,Restaurant,Sandwich Place
1,Bayview Village,Bank,Skating Rink,Grocery Store,Chinese Restaurant,Café,Japanese Restaurant,Skate Park,Restaurant,Dance Studio,Department Store
2,Bedford Park,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Juice Bar,Pub,Butcher,Pizza Place,Pharmacy,Café,Liquor Store
3,CFB Toronto,Airport,Park,Coffee Shop,Sandwich Place,Women's Store,Dim Sum Restaurant,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store
4,Downsview West,Shopping Mall,Park,Vietnamese Restaurant,Pizza Place,Grocery Store,Coffee Shop,Moving Target,Bank,Convenience Store,Dance Studio


In [49]:
#import k-means from clustering stage
from sklearn.cluster import KMeans
ny_data = borough_data.drop(16)
# set number of clusters
kclusters = 4

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 2, 1, 1, 1, 1, 1], dtype=int32)

In [50]:
len(kmeans.labels_)#=19

19

In [62]:
ny_merged = borough_data

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,Borough,Neighborhood,Postcode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Victoria Village,M4A,43.725882,-79.315572,2,Park,Portuguese Restaurant,Sporting Goods Shop,Coffee Shop,Café,Hockey Arena,Playground,Pizza Place,Deli / Bodega,Clothing Store
1,North York,"Silver Hills, York Mills",M2L,43.757490,-79.374714,1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio
2,North York,Bayview Village,M2K,43.786947,-79.385975,1,Bank,Skating Rink,Grocery Store,Chinese Restaurant,Café,Japanese Restaurant,Skate Park,Restaurant,Dance Studio,Department Store
3,North York,Bedford Park,M5M,43.733283,-79.419750,2,Coffee Shop,Italian Restaurant,Fast Food Restaurant,Juice Bar,Pub,Butcher,Pizza Place,Pharmacy,Café,Liquor Store
4,North York,Downsview West,M3L,43.739015,-79.506944,2,Shopping Mall,Park,Vietnamese Restaurant,Pizza Place,Grocery Store,Coffee Shop,Moving Target,Bank,Convenience Store,Dance Studio


In [63]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']
